### Libraries

In [11]:
import pandas as pd
import numpy as np
from pprint import pprint

import warnings
import os

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV, KFold

import matplotlib.pyplot as plt

In [12]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")
os.chdir("C:\\Users\\joaos\\Documents\\GitHub\\pred_se_ml\data")

##### Importing the data

In [13]:
# Importing the data
df = pd.read_csv("pred_se_ml.csv")

df.shape

(2420, 26)

In [14]:
df.head()

,country,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden,diversity,quality,se_medina,democ_closed_anocracy,democ_democracy,democ_full_democracy,democ_interregnum,democ_interruption,democ_open_anocracy,democ_transition,lgdp_pc
0,angola,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,transition,61.6,6.002369,0.37,52.47,0,0,0,0,0,0,1,5.986754
1,angola,1996,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,454.375004,NaN,transition,54.6,6.048760,0.40,46.30,0,0,0,0,0,0,1,6.118923
2,angola,1997,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,516.127849,NaN,autocracy,52.6,5.991146,0.35,50.48,0,0,0,0,0,0,0,6.246355
3,angola,1998,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,423.403332,NaN,autocracy,59.1,5.882152,0.34,49.21,0,0,0,0,0,0,0,6.048325
4,angola,1999,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,387.689415,NaN,autocracy,47.9,5.903543,0.34,48.64,0,0,0,0,0,0,0,5.960205


In [15]:
# Missing data by column (%)
df.isnull().mean().sort_values(ascending=False)

tribute_time             0.536777
property_time            0.495868
cost_procedures          0.455372
business_time            0.455372
business_procedure       0.455372
unemployment             0.335950
governement_spending     0.078926
exchange                 0.070661
tax_burden               0.046694
quality                  0.011570
inflation                0.007438
fdi                      0.006198
lgdp_pc                  0.002066
gdp_pc                   0.002066
democ_transition         0.000000
democ_open_anocracy      0.000000
democ_full_democracy     0.000000
democ_interruption       0.000000
democ_interregnum        0.000000
country                  0.000000
democ_democracy          0.000000
democ_closed_anocracy    0.000000
se_medina                0.000000
diversity                0.000000
year                     0.000000
democ                    0.000000
dtype: float64

In [16]:
# df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2420 entries, 0 to 2419
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                2420 non-null   object 
 1   year                   2420 non-null   int64  
 2   fdi                    2405 non-null   float64
 3   property_time          1220 non-null   float64
 4   cost_procedures        1318 non-null   float64
 5   business_time          1318 non-null   float64
 6   business_procedure     1318 non-null   float64
 7   tribute_time           1121 non-null   float64
 8   governement_spending   2229 non-null   float64
 9   exchange               2249 non-null   float64
 10  inflation              2402 non-null   float64
 11  gdp_pc                 2415 non-null   float64
 12  unemployment           1607 non-null   float64
 13  democ                  2420 non-null   object 
 14  tax_burden             2307 non-null   float64
 15  dive

There is already dummy columns regarding democracy, hence, the democ column will be dropped

In [17]:
df = df.drop('democ', axis=1)

### Dealing with missing data

There is a lot of missing data, especially in relation to the tribute_time, however, if we limit the year range from 2004 to 2014, the proportion of missing data will decrease

In [18]:
df = df[(df['year']>=2004) & (df['year'] <= 2014)]

In [19]:
df.isnull().mean().sort_values(ascending=False)*100

unemployment             29.000751
tribute_time             15.777611
property_time             8.339594
cost_procedures           8.339594
business_time             8.339594
business_procedure        8.339594
governement_spending      6.235913
exchange                  5.785124
tax_burden                1.051841
democ_closed_anocracy     0.000000
democ_transition          0.000000
democ_open_anocracy       0.000000
democ_interruption        0.000000
democ_interregnum         0.000000
democ_full_democracy      0.000000
democ_democracy           0.000000
country                   0.000000
se_medina                 0.000000
quality                   0.000000
diversity                 0.000000
year                      0.000000
gdp_pc                    0.000000
inflation                 0.000000
fdi                       0.000000
lgdp_pc                   0.000000
dtype: float64

In [20]:
# Proportion of missing data by country in the column 'unemployment' since it's the variable with the biggest proportion of missing data
missing_by_country = df.groupby('country')['unemployment'].apply(lambda x: x.isnull().sum() / x.count()).sort_values(ascending=False)
#print(missing_by_country)

# Collecting the names of countries that have a proportion of missing data in the unemployment column above 5%
countries = missing_by_country[missing_by_country > 5].index.tolist()
#countries

# Removing these countries
df = df[~df['country'].isin(countries)]

In [21]:
# Replacing missing values ​​with the average for each country for each attribute
mean_by_country = df.groupby('country').transform('mean')
df.fillna(mean_by_country, inplace=True)

After the imputation, 'government spending' and 'exchange' still having missing values, this happens because some countries have the 'inf' as observations and not a usual 

In [22]:
# This is seen with the follow code for governement_spending and after for exchange (just change governement_spending by exchange to see all the countries)
missing_by_country = df.groupby('country')['governement_spending'].apply(lambda x: x.isnull().sum() / x.count()).sort_values(ascending=False)
#print(missing_by_country)

In [23]:
# Droping these countries
countries_to_drop = ['malawi', 'trinidad_and_tobago', 'liberia', 'yemen', "cote_d'ivoire", "nigeria"]
df = df[~df['country'].isin(countries_to_drop)]

In [24]:
# No more missing values
df.isnull().sum().sort_values(ascending=False)

country                  0
tax_burden               0
democ_transition         0
democ_open_anocracy      0
democ_interruption       0
democ_interregnum        0
democ_full_democracy     0
democ_democracy          0
democ_closed_anocracy    0
se_medina                0
quality                  0
diversity                0
unemployment             0
year                     0
gdp_pc                   0
inflation                0
exchange                 0
governement_spending     0
tribute_time             0
business_procedure       0
business_time            0
cost_procedures          0
property_time            0
fdi                      0
lgdp_pc                  0
dtype: int64

In [25]:
# Droping columns that will not be used
df = df.drop(['country', 'year'], axis=1)

In [26]:
df.head()

,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,tax_burden,diversity,quality,se_medina,democ_closed_anocracy,democ_democracy,democ_full_democracy,democ_interregnum,democ_interruption,democ_open_anocracy,democ_transition,lgdp_pc
9,9.329239,335.0,910.0,83.0,12.0,289.0,14.312063,103.579947,33.443595,1254.696126,23.6430,84.916689,6.330621,0.40,46.81,1,0,0,0,0,0,0,7.134649
10,-3.526657,335.0,653.8,83.0,12.0,284.0,16.029037,106.590962,42.374249,1900.723816,12.6408,84.916689,6.245515,0.42,43.84,1,0,0,0,0,0,0,7.549990
11,-0.072001,335.0,498.2,83.0,12.0,284.0,15.341722,94.625159,17.115665,2597.963585,12.6408,84.900000,6.295336,0.40,41.23,1,0,0,0,0,0,0,7.862483
12,-1.368762,335.0,343.7,83.0,12.0,284.0,15.536935,108.060068,4.308432,3121.348735,12.6408,85.000000,6.211946,0.45,37.13,1,0,0,0,0,0,0,8.046020
13,1.896314,335.0,196.8,68.0,8.0,284.0,16.814612,121.364708,19.365774,4081.717497,12.6408,85.175100,6.315526,0.47,35.26,1,0,0,0,0,0,0,8.314273


In [27]:
# Renaming the columns for better future visualization
rename_dictionary = {'se_medina':'Shadow Economy',
                        'lgdp_pc': 'Log. GDP Per Capita', 
                         'democ_transition': 'Dem. Transition', 
                         'democ_interruption': 'Dem. Interruption',
                        'democ_interregnum':'Dem. Interregnum',
                        'democ_full_democracy': 'Dem. Full',
                        'democ_democracy':'Democracy',
                        'democ_autocracy':'Autocracy',
                        'democ_closed_anocracy':'Closed Anocracy',
                        'democ_open_anocracy': 'Open Anocracy',
                        'quality':'Quality',
                        'diversity':'Diversity',
                        'tax_burden':'Tax Burden',
                        'unemployment':'Unemployment',
                        'inflation':'Inflation',
                        'exchange':'Exchange',
                        'governement_spending':'Government Spending',
                         'tribute_time':'Tribute Time',
                        'business_procedure':'Business Procedure',
                        'business_time':'Business Time',
                        'cost_procedures':'Cost Procedures',
                        'property_time':'Property Time',
                        'fdi':'FDI'}

df = df.rename(columns=rename_dictionary)

In [28]:
# Changing the order
new_order = ['Shadow Economy', 'FDI', 'Property Time', 'Cost Procedures', 'Business Time', 'Business Procedure',
               'Tribute Time', 'Government Spending', 'Exchange', 'Inflation', 'Unemployment', 'Tax Burden', 
               'Diversity', 'Quality', 'Closed Anocracy', 'Democracy', 'Dem. Full', 'Dem. Interregnum', 
               'Dem. Interruption', 'Open Anocracy', 'Dem. Transition', 'Log. GDP Per Capita']

df = df.reindex(columns=new_order)

### Data padronization

In [29]:
# Save the columns' names
columns_names = df.columns

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply the MinMax
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=columns_names)

### Spliting between train and test

In [30]:
train_set, test_set = train_test_split(df_scaled, test_size=0.2, random_state=0)

### Spliting the data between train and validation

In [31]:
# separating the data into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(
    train_set.drop(labels=['Shadow Economy'], axis=1),
    train_set['Shadow Economy'],
    test_size=0.2,
    random_state=0
)

### Regressions

#### Linear regression

In [41]:
from sklearn.linear_model import LinearRegression

# Params set
linear_param_grid = {
    'fit_intercept': [True, False],
    'copy_X':[True, False]
}

# Create a linear regression model
linear_model = LinearRegression()

# Perform Grid Search with cross-validation
linear_grid_search = GridSearchCV(linear_model, 
                                  linear_param_grid, 
                                  scoring='neg_mean_absolute_percentage_error', 
                                  cv=5,
                                  verbose=2,
                                  n_jobs=-1)

# Fitting the model
linear_grid_search.fit(X_train, y_train)

# Print the best parameters
print("\nBest Parameters: ", linear_grid_search.best_params_)

# Get the best model
linear_best_model = linear_grid_search.best_estimator_

# Predictions and evaluations on validation set
y_pred = linear_best_model.predict(X_valid)
mape = np.mean(np.abs((y_valid - y_pred) / y_valid)) * 100
print(f'\nMean Absolute Percentage Error (MAPE) on validation set: {mape:.2f}%')

# Predictions and evaluations on test set
y_pred_test = linear_best_model.predict(test_set.drop(labels=['Shadow Economy'], axis=1))
mape_test = np.mean(np.abs((test_set['Shadow Economy'] - y_pred_test) / test_set['Shadow Economy'])) * 100
print(f'Mean Absolute Percentage Error (MAPE) on test set: {mape_test:.2f}%')

Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best Parameters:  {'copy_X': True, 'fit_intercept': False}

Mean Absolute Percentage Error (MAPE) on validation set: 52.03%
Mean Absolute Percentage Error (MAPE) on test set: 40.06%
